In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ib_insync import *
import pandas as pd
from IPython.display import display, clear_output
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from keras.src.callbacks import EarlyStopping
import random
import numpy as np
import tensorflow as tf
from keras.api.layers import Dense, Dropout
from keras.api.models import Sequential
from keras.api.regularizers import l1, l2
from keras.api.optimizers import Adam
util.startLoop()

In [14]:
import keras
import pickle

model = keras.models.load_model('daily_v62.keras')
params = pickle.load(open('params_dailyv62.pkl', 'rb'))
mu = params['mu']
std = params['std']
features = params['features']

In [ ]:
ib = IB()
connection = ib.connect()

In [8]:
def prepare_dataframe():
    symbol = "EURUSD=X"
    start = "2004-01-01"
    end = "2024-08-26"
    interval = '1d'
    
    df = yf.download(symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    df.drop(columns=['Adj Close', 'Volume'], inplace=True)
    
    ##INSERT TODAY
    df.loc[pd.to_datetime('today').normalize()] = np.nan
    ####
    
    
    df['returns'] = np.log(df['Close'].div(df['Close'].shift(1)))
    df['day_of_week'] = df.index.dayofweek
    df['dir'] = np.where(df['returns'] > 0, 1,0)
    #df.dropna(inplace=True)
    
    tnx_symbol = "^TNX"
    
    tnx = yf.download(tnx_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    tnx.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    #df.index = df.index.tz_convert('utc')
    tnx.rename(columns={'Close': 'tnx_close', 'Volume': 'tnx_volume'}, inplace=True)
    tnx['tnx_returns'] = np.log(tnx['tnx_close'].div(tnx['tnx_close'].shift(1)))
    #tnx['tnx_simple_returns'] = tnx['tnx_close'] - tnx['tnx_close'].shift(1)
    tnx.dropna(inplace=True)
    forex_combined = tnx.join(df, how='outer')
    
    snp_symbol = "^SPX"
    
    snp = yf.download(snp_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    snp.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    #df.index = df.index.tz_convert('utc')
    snp.rename(columns={'Close': 'snp_close', 'Volume': 'snp_volume'}, inplace=True)
    snp['snp_returns'] = np.log(snp['snp_close'].div(snp['snp_close'].shift(1)))
    snp.dropna(inplace=True)
    forex_combined = snp.join(forex_combined, how='outer')
    
    gold_symbol = "GC=F"
    
    gold = yf.download(gold_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    gold.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    #df.index = df.index.tz_convert('utc')
    gold.rename(columns={'Close': 'gold_close', 'Volume': 'gold_volume'}, inplace=True)
    gold['gold_returns'] = np.log(gold['gold_close'].div(gold['gold_close'].shift(1)))
    gold.dropna(inplace=True)
    forex_combined = gold.join(forex_combined, how='outer')
    
    oil_symbol = "CL=F"
    
    oil = yf.download(oil_symbol, start, end, interval=interval)
    #df.rename(columns={"Close": "price"}, inplace=True)
    oil.drop(columns=['Adj Close', 'Open', 'High', 'Low'], inplace=True)
    oil.rename(columns={'Close': 'oil_close', 'Volume': 'oil_volume'}, inplace=True)
    oil['oil_returns'] = np.log(oil['oil_close'].div(oil['oil_close'].shift(1)))
    oil.dropna(inplace=True)
    forex_combined = oil.join(forex_combined, how='outer')
    
    import seaborn as sns
    %matplotlib inline
    
    forex_clean = forex_combined.copy()
    forex_clean.drop(columns=['Open', 'High', 'Low'], inplace=True)
    
    features = ['Close', 'returns', 'oil_returns', 'snp_returns', 'tnx_returns', 'gold_returns', 'gold_volume', 'snp_volume', 'tnx_volume',]
    cols = []
    for f in features:
        for lag in range(1, 7 + 1):
            col = "{}_lag_{}".format(f, lag)
            forex_clean[col] = forex_clean[f].shift(lag)
            cols.append(col)
   # forex_clean.dropna(inplace=True)
    
    df = forex_clean.copy()
    
    return df

In [9]:
df = prepare_dataframe()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
C:\code\trading\v1\pythonProject\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,oil_close,oil_volume,oil_returns,gold_close,gold_volume,gold_returns,snp_close,snp_volume,snp_returns,tnx_close,...,snp_volume_lag_5,snp_volume_lag_6,snp_volume_lag_7,tnx_volume_lag_1,tnx_volume_lag_2,tnx_volume_lag_3,tnx_volume_lag_4,tnx_volume_lag_5,tnx_volume_lag_6,tnx_volume_lag_7
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,1122.219971,1.578200e+09,0.012319,4.387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-06,33.700001,103405.0,-0.002371,422.799988,20.0,-0.003777,1123.670044,1.494500e+09,0.001291,4.277,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-07,33.619999,81675.0,-0.002377,421.899994,20.0,-0.002131,1126.329956,1.704900e+09,0.002364,4.246,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-21,71.930000,390527.0,-0.028912,2508.399902,96.0,-0.001156,5620.850098,3.119670e+09,0.004231,3.778,...,3.380050e+09,3.648980e+09,3.360160e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-08-22,73.010002,334117.0,0.014903,2478.899902,210.0,-0.011830,5570.640137,2.981530e+09,-0.008973,3.862,...,3.723310e+09,3.380050e+09,3.648980e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-08-23,74.830002,334117.0,0.024622,2508.399902,210.0,0.011830,5634.609863,3.291940e+09,0.011418,3.807,...,3.357690e+09,3.723310e+09,3.380050e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_standardized = (df - mu) / std
df_standardized

,oil_close,oil_volume,oil_returns,gold_close,gold_volume,gold_returns,snp_close,snp_volume,snp_returns,tnx_close,...,snp_volume_lag_5,snp_volume_lag_6,snp_volume_lag_7,tnx_volume_lag_1,tnx_volume_lag_2,tnx_volume_lag_3,tnx_volume_lag_4,tnx_volume_lag_5,tnx_volume_lag_6,tnx_volume_lag_7
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,-0.953288,-1.550414,0.951454,1.328098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-06,-1.580876,-1.102227,-0.104855,-1.816665,-0.173048,-0.340781,-0.951059,-1.611785,0.094079,1.227024,...,NaN,NaN,NaN,-0.104027,NaN,NaN,NaN,NaN,NaN,NaN
2004-01-07,-1.584452,-1.203835,-0.105080,-1.818982,-0.173048,-0.200184,-0.946972,-1.457514,0.177505,1.198539,...,NaN,NaN,NaN,-0.104027,-0.104027,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-21,0.128095,0.240337,-1.143655,3.552944,-0.169944,-0.116880,5.959926,-0.420172,0.322607,0.768516,...,-0.250624,-0.042544,-0.242713,-0.104027,-0.104027,-0.104027,-0.104027,-0.102372,-0.104027,-0.104028
2024-08-22,0.176374,-0.023432,0.571239,3.476993,-0.165288,-1.028544,5.882767,-0.521459,-0.703930,0.845700,...,0.005290,-0.243979,-0.026494,-0.104027,-0.104027,-0.104027,-0.104027,-0.102372,-0.104027,-0.104028
2024-08-23,0.257732,-0.023432,0.951652,3.552944,-0.165288,0.992155,5.981071,-0.293859,0.881388,0.795162,...,-0.267294,0.013131,-0.227823,-0.104027,-0.104027,-0.104027,-0.104027,-0.102372,-0.104027,-0.104028


In [16]:
df["probability"] = model.predict(df_standardized[features])
df

169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,oil_close,oil_volume,oil_returns,gold_close,gold_volume,gold_returns,snp_close,snp_volume,snp_returns,tnx_close,...,snp_volume_lag_6,snp_volume_lag_7,tnx_volume_lag_1,tnx_volume_lag_2,tnx_volume_lag_3,tnx_volume_lag_4,tnx_volume_lag_5,tnx_volume_lag_6,tnx_volume_lag_7,probability
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.496664
2004-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.496664
2004-01-05,NaN,NaN,NaN,NaN,NaN,NaN,1122.219971,1.578200e+09,0.012319,4.387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.496664
2004-01-06,33.700001,103405.0,-0.002371,422.799988,20.0,-0.003777,1123.670044,1.494500e+09,0.001291,4.277,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.496664
2004-01-07,33.619999,81675.0,-0.002377,421.899994,20.0,-0.002131,1126.329956,1.704900e+09,0.002364,4.246,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.496664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-21,71.930000,390527.0,-0.028912,2508.399902,96.0,-0.001156,5620.850098,3.119670e+09,0.004231,3.778,...,3.648980e+09,3.360160e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.487512
2024-08-22,73.010002,334117.0,0.014903,2478.899902,210.0,-0.011830,5570.640137,2.981530e+09,-0.008973,3.862,...,3.380050e+09,3.648980e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.519170
2024-08-23,74.830002,334117.0,0.024622,2508.399902,210.0,0.011830,5634.609863,3.291940e+09,0.011418,3.807,...,3.723310e+09,3.380050e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.511000
